here, we will test various time series classification methods. We choose **three** datasets from [the UCR/UEA time series repository](http://timeseriesclassification.com) and perform the tasks by evaluating the models on three selected datasets.



### Task 0: Preparation

choosing **three** datasets from the UCR/UEA time series repository.

In [ ]:
!pip install sktime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 37.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
from sktime.datasets import load_UCR_UEA_dataset

In [ ]:
#First dataset
X_train_Pro, y_train_Pro = load_UCR_UEA_dataset(name='ProximalPhalanxTW', split='train', return_type="numpy2D")
X_test_Pro , y_test_Pro = load_UCR_UEA_dataset(name='ProximalPhalanxTW', split='test', return_type="numpy2D")


In [ ]:
X_train_Pro.shape , y_train_Pro.shape, X_test_Pro.shape, y_test_Pro.shape

((400, 80), (400,), (205, 80), (205,))

In [ ]:
labels = np.unique(y_train_Pro).tolist()
labels

['3', '4', '5', '6', '7', '8']

In [ ]:
#one_hot_encode the labels

# Define the labels
labels = np.unique(y_train_Pro).tolist()

# One-hot encode the labels
label_map = {label: i for i, label in enumerate(labels)}
# Convert the labels to integers using the label map
y_train_Pro_encoded = [label_map[label] for label in y_train_Pro]
y_test_Pro_encoded = [label_map[label] for label in y_test_Pro]
# Perform one-hot encoding using TensorFlow
y_train_Pro_encoded = tf.one_hot(y_train_Pro_encoded, len(labels))
y_test_Pro_encoded = tf.one_hot(y_test_Pro_encoded, len(labels))

y_train_Pro_encoded = y_train_Pro_encoded.numpy()
y_test_Pro_encoded = y_test_Pro_encoded.numpy()

In [ ]:
y_train_Pro_encoded.shape, type(y_train_Pro_encoded), y_test_Pro_encoded.shape

((400, 6), numpy.ndarray, (205, 6))

In [ ]:
y_train_Pro_encoded

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
# The second dataset
X_train_Plane, y_train_Plane = load_UCR_UEA_dataset(name='Plane', split='train', return_type="numpy2D")
X_test_Plane , y_test_Plane = load_UCR_UEA_dataset(name='Plane', split='test', return_type="numpy2D")

In [ ]:
X_train_Plane.shape , y_train_Plane.shape,X_test_Plane.shape, y_test_Plane.shape

((105, 144), (105,), (105, 144), (105,))

In [ ]:
np.unique(y_train_Plane), type(y_train_Plane)

(array(['1', '2', '3', '4', '5', '6', '7'], dtype='<U1'), numpy.ndarray)

In [ ]:
#one_hot_encode the labels for the second dataset

# Define the labels
labels = np.unique(y_train_Plane).tolist()

# # One-hot encode the labels
label_map = {label: i for i, label in enumerate(labels)}
# Convert the labels to integers using the label map
y_train_Plane_encoded = [label_map[label] for label in y_train_Plane]
y_test_Plane_encoded = [label_map[label] for label in y_test_Plane]
# Perform one-hot encoding using TensorFlow
y_train_Plane_encoded = tf.one_hot(y_train_Plane_encoded, len(labels))
y_test_Plane_encoded = tf.one_hot(y_test_Plane_encoded, len(labels))

y_train_Plane_encoded = y_train_Plane_encoded.numpy()
y_test_Plane_encoded = y_test_Plane_encoded.numpy()

In [ ]:
#The third dataset
#ECG200
X_train_ECG, y_train_ECG = load_UCR_UEA_dataset(name='ECG200', split='train', return_type="numpy2D")
X_test_ECG , y_test_ECG = load_UCR_UEA_dataset(name='ECG200', split='test', return_type="numpy2D")

In [ ]:
X_train_ECG.shape , y_train_ECG.shape,X_test_ECG.shape, y_test_ECG.shape

((100, 96), (100,), (100, 96), (100,))

In [ ]:
np.unique(y_train_ECG), type(y_train_ECG)

(array(['-1', '1'], dtype='<U2'), numpy.ndarray)

In [ ]:
#one_hot_encode the labels for the third dataset

# Define the labels
labels = np.unique(y_train_ECG).tolist()
# One-hot encode the labels
label_map = {label: i for i, label in enumerate(labels)}
# Convert the labels to integers using the label map
y_train_ECG_encoded = [label_map[label] for label in y_train_ECG]
y_test_ECG_encoded = [label_map[label] for label in y_test_ECG]
# Perform one-hot encoding using TensorFlow
y_train_ECG_encoded = tf.one_hot(y_train_ECG_encoded, len(labels))
y_test_ECG_encoded = tf.one_hot(y_test_ECG_encoded, len(labels))

y_train_ECG_encoded = y_train_ECG_encoded.numpy()
y_test_ECG_encoded = y_test_ECG_encoded.numpy()

### Task 1: Time series classification using deep learning 1



Time series classification problems can be solved using networks like CNN, RNN, or FCN. We try to merge different networks. 

Try to implement four different models: 1) Fully connected network, 2) **[Bi-directional RNN](https://keras.io/api/layers/recurrent_layers/bidirectional/)**, 3) 1D-CNN only, and 4) 1D-CNN + Any RNN variant. Here it is thr following rules:

- When initially loading the dataset, use sktime's `load_UCR_UEA_dataset` function. This is for our grading purpose.
- We use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- We use Tensorflow's data API (`tf.data`) to manage dataset and use `shuffle`, `batch,` and `prefetch` functions. This means that we need to convert the data format using the `from_tensor_slices` function. This also means that we need to create your own validation set. we are not limited to using any methods to do this, but we may also need to shuffle the dataset before (for that, you can use `np.random.permutation`).
- We need to clearly report the **test accuracy** of the four models. Training and validation accuracy scores are not enough.
- We need to deal with datasets of different sizes. In this case, it might be helpful to make a function to create a model that can receive different input sizes as a parameter.


In [ ]:
# Two Tensor flow callbacks
checkpoint = keras.callbacks.ModelCheckpoint("checkpoint")
# checkpoint_cb = keras.callbacks.ModelCheckpoint("checkpoint", save_best_only=True)

# count the number of epochs with the loss higher than the current lowest
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)


In [ ]:
#create test,valid,train datasets using from_tensor_slice for the first dataset
train_ds_Pro = tf.data.Dataset.from_tensor_slices((X_train_Pro, y_train_Pro_encoded))  
# Shuffle the training data
idx = np.random.permutation(len(X_train_Pro))
X_train_Pro, y_train_Pro_encoded = X_train_Pro[idx], y_train_Pro_encoded[idx] 
train_ds_Pro = train_ds_Pro.map(lambda x, y: (tf.cast(x, tf.float32), y))
# create train and valid datasets
valid_split = 0.2
valid_size = int(valid_split * len(X_train_Pro))
train_ds_Pro = tf.data.Dataset.from_tensor_slices((X_train_Pro[valid_size:], y_train_Pro_encoded[valid_size:])) 
valid_ds_Pro = tf.data.Dataset.from_tensor_slices((X_train_Pro[:valid_size], y_train_Pro_encoded[:valid_size]))
#create test dataset
test_ds_Pro = tf.data.Dataset.from_tensor_slices((X_test_Pro, y_test_Pro_encoded)) 
test_ds_Pro = test_ds_Pro.map(lambda x, y: (tf.cast(x, tf.float32), y))

#manage dataset and use shuffle, batch, and prefetch functions
train_ds_Pro = train_ds_Pro.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
valid_ds_Pro = valid_ds_Pro.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds_Pro = test_ds_Pro.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_ds_Pro.cardinality(), valid_ds_Pro.cardinality(),  test_ds_Pro.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=10>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=7>)

In [ ]:
#create test,valid,train datasets using from_tensor_slice for the second dataset
train_ds_Plane = tf.data.Dataset.from_tensor_slices((X_train_Plane, y_train_Plane_encoded))
# Shuffle the training data
idx = np.random.permutation(len(X_train_Plane))
X_train_Plane, y_train_Plane_encoded = X_train_Plane[idx], y_train_Plane_encoded[idx]
train_ds_Plane = train_ds_Plane.map(lambda x, y: (tf.cast(x, tf.float32), y))
# create train and valid datasets
valid_split = 0.2
valid_size = int(valid_split * len(X_train_Plane))
train_ds_Plane = tf.data.Dataset.from_tensor_slices((X_train_Plane[valid_size:], y_train_Plane_encoded[valid_size:]))
valid_ds_Plane = tf.data.Dataset.from_tensor_slices((X_train_Plane[:valid_size], y_train_Plane_encoded[:valid_size]))
#creat test dataset
test_ds_Plane = tf.data.Dataset.from_tensor_slices((X_test_Plane, y_test_Plane_encoded))
test_ds_Plane = test_ds_Plane.map(lambda x, y: (tf.cast(x, tf.float32), y))

#manage dataset and use shuffle, batch, and prefetch functions
train_ds_Plane = train_ds_Plane.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
valid_ds_Plane = valid_ds_Plane.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds_Plane = test_ds_Plane.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
#create test,valid,train datasets using from_tensor_slice for the third dataset
train_ds_ECG = tf.data.Dataset.from_tensor_slices((X_train_ECG, y_train_ECG_encoded))
# Shuffle the training data
idx = np.random.permutation(len(X_train_ECG))
X_train_ECG, y_train_ECG_encoded = X_train_ECG[idx], y_train_ECG_encoded[idx]
train_ds_ECG = train_ds_ECG.map(lambda x, y: (tf.cast(x, tf.float32), y))
# create train and valid datasets
valid_split = 0.2
valid_size = int(valid_split * len(X_train_Plane))
train_ds_ECG = tf.data.Dataset.from_tensor_slices((X_train_ECG[valid_size:], y_train_ECG_encoded[valid_size:]))
valid_ds_ECG = tf.data.Dataset.from_tensor_slices((X_train_ECG[:valid_size], y_train_ECG_encoded[:valid_size]))
#creat test dataset
test_ds_ECG = tf.data.Dataset.from_tensor_slices((X_test_ECG, y_test_ECG_encoded))
test_ds_ECG = test_ds_ECG.map(lambda x, y: (tf.cast(x, tf.float32), y))

#manage dataset and use shuffle, batch, and prefetch functions
train_ds_ECG = train_ds_ECG.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
valid_ds_ECG = valid_ds_ECG.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds_ECG = test_ds_ECG.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
#Implemeting four different models

In [ ]:
#fully connected network
def create_FCN(input_shape, num_labels):
    model = keras.models.Sequential([
    keras.layers.Dense(300, input_shape=[input_shape], activation='relu', name='dense1'),
    keras.layers.Dense(100, activation='relu', name='dense2'),
    keras.layers.Dense(num_labels, activation='sigmoid', name='output')
])
    
    model.compile(loss="categorical_crossentropy", 
                  optimizer="sgd", 
                  metrics=["accuracy"])
    return model


In [ ]:
#Implementing fully connected network on the datasets
model_FCN = create_FCN(80,6)
history = model_FCN.fit(x= train_ds_Pro, validation_data= valid_ds_Pro, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
 1/10 [==>...........................] - ETA: 6s - loss: 2.2310 - accuracy: 0.0625

10/10 [==============================] - 2s 163ms/step - loss: 1.6463 - accuracy: 0.3844 - val_loss: 1.4744 - val_accuracy: 0.4250
Epoch 2/10
 9/10 [==========================>...] - ETA: 0s - loss: 1.3541 - accuracy: 0.4479

10/10 [==============================] - 1s 133ms/step - loss: 1.3421 - accuracy: 0.4563 - val_loss: 1.4303 - val_accuracy: 0.4250
Epoch 3/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.1590 - accuracy: 0.5312

10/10 [==============================] - 1s 86ms/step - loss: 1.2658 - accuracy: 0.4563 - val_loss: 1.4013 - val_accuracy: 0.4250
Epoch 4/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.2106 - accuracy: 0.3438

10/10 [==============================] - 1s 85ms/step - loss: 1.2136 - accuracy: 0.4875 - val_loss: 1.3754 - val_accuracy: 0.4250
Epoch 5/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.2459 - accuracy: 0.3750

...

10/10 [==============================] - 1s 86ms/step - loss: 1.1295 - accuracy: 0.5344 - val_loss: 1.3156 - val_accuracy: 0.5500
Epoch 7/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.2298 - accuracy: 0.5000

10/10 [==============================] - 1s 86ms/step - loss: 1.0967 - accuracy: 0.6156 - val_loss: 1.3016 - val_accuracy: 0.6000
Epoch 8/10
 1/10 [==>...........................] - ETA: 0s - loss: 0.9455 - accuracy: 0.6562

10/10 [==============================] - 2s 174ms/step - loss: 1.0599 - accuracy: 0.6750 - val_loss: 1.2760 - val_accuracy: 0.5375
Epoch 9/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.0333 - accuracy: 0.5938

10/10 [==============================] - 1s 92ms/step - loss: 1.0373 - accuracy: 0.6500 - val_loss: 1.2426 - val_accuracy: 0.6625
Epoch 10/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.0321 - accuracy: 0.7500

10/10 [==============================] - 1s 88ms/step - loss: 1.0054 - accuracy: 0.6844 - val_loss: 1.2287 - val_accuracy: 0.6750


In [ ]:
model_FCN.evaluate(test_ds_Pro)

7/7 [==============================] - 0s 5ms/step - loss: 1.1072 - accuracy: 0.6780


[1.1072337627410889, 0.6780487895011902]

In [ ]:
X_train_Plane.shape, train_ds_Plane.cardinality()

((105, 144), <tf.Tensor: shape=(), dtype=int64, numpy=3>)

In [ ]:
model_FCN = create_FCN(144,7)
history = model_FCN.fit(train_ds_Plane, validation_data=valid_ds_Plane, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
1/3 [=========>....................] - ETA: 1s - loss: 2.6093 - accuracy: 0.1562

3/3 [==============================] - 2s 636ms/step - loss: 2.2840 - accuracy: 0.1786 - val_loss: 1.7640 - val_accuracy: 0.2857
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 1.8614 - accuracy: 0.1562

3/3 [==============================] - 1s 542ms/step - loss: 1.6801 - accuracy: 0.4167 - val_loss: 1.5187 - val_accuracy: 0.5714
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 1.3670 - accuracy: 0.6875

3/3 [==============================] - 1s 596ms/step - loss: 1.3815 - accuracy: 0.6429 - val_loss: 1.3530 - val_accuracy: 0.5238
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 1.1965 - accuracy: 0.6250

3/3 [==============================] - 1s 358ms/step - loss: 1.2280 - accuracy: 0.6310 - val_loss: 1.2248 - val_accuracy: 0.5714
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 1.1465 - accuracy: 0.5938

...

3/3 [==============================] - 1s 384ms/step - loss: 1.0223 - accuracy: 0.6905 - val_loss: 1.0114 - val_accuracy: 0.5714
Epoch 7/10
1/3 [=========>....................] - ETA: 0s - loss: 0.9639 - accuracy: 0.6875

3/3 [==============================] - 1s 364ms/step - loss: 0.9378 - accuracy: 0.7262 - val_loss: 0.8845 - val_accuracy: 0.8095
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 0.9556 - accuracy: 0.9062

3/3 [==============================] - 1s 389ms/step - loss: 0.8774 - accuracy: 0.9167 - val_loss: 0.8494 - val_accuracy: 0.6667
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 0.7971 - accuracy: 0.8125

3/3 [==============================] - 1s 406ms/step - loss: 0.8040 - accuracy: 0.8333 - val_loss: 0.7906 - val_accuracy: 0.7143
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 0.7785 - accuracy: 0.8438

3/3 [==============================] - 1s 374ms/step - loss: 0.7515 - accuracy: 0.8095 - val_loss: 0.7023 - val_accuracy: 1.0000


In [ ]:
model_FCN.evaluate(test_ds_Plane)

4/4 [==============================] - 0s 5ms/step - loss: 0.7844 - accuracy: 0.9238


[0.784355878829956, 0.9238095283508301]

In [ ]:
X_train_ECG.shape, train_ds_ECG.cardinality()

((100, 96), <tf.Tensor: shape=(), dtype=int64, numpy=3>)

In [ ]:
model_FCN = create_FCN(96,2)
history = model_FCN.fit(train_ds_ECG, validation_data=valid_ds_ECG, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
1/3 [=========>....................] - ETA: 2s - loss: 0.6459 - accuracy: 0.6562

3/3 [==============================] - 2s 465ms/step - loss: 0.6214 - accuracy: 0.6962 - val_loss: 0.6304 - val_accuracy: 0.6667
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 0.6770 - accuracy: 0.5938

3/3 [==============================] - 1s 391ms/step - loss: 0.5673 - accuracy: 0.6962 - val_loss: 0.5690 - val_accuracy: 0.6667
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5940 - accuracy: 0.6250

3/3 [==============================] - 1s 380ms/step - loss: 0.5295 - accuracy: 0.6962 - val_loss: 0.5234 - val_accuracy: 0.7619
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4701 - accuracy: 0.8125

3/3 [==============================] - 1s 522ms/step - loss: 0.4986 - accuracy: 0.7722 - val_loss: 0.4823 - val_accuracy: 0.8095
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5125 - accuracy: 0.7500

...

3/3 [==============================] - 1s 565ms/step - loss: 0.4549 - accuracy: 0.7848 - val_loss: 0.4172 - val_accuracy: 0.8095
Epoch 7/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4807 - accuracy: 0.7812

3/3 [==============================] - 1s 357ms/step - loss: 0.4407 - accuracy: 0.7722 - val_loss: 0.3960 - val_accuracy: 0.8095
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 0.3617 - accuracy: 0.8750

3/3 [==============================] - 1s 350ms/step - loss: 0.4285 - accuracy: 0.7848 - val_loss: 0.3691 - val_accuracy: 0.8095
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4900 - accuracy: 0.7188

3/3 [==============================] - 1s 359ms/step - loss: 0.4133 - accuracy: 0.7848 - val_loss: 0.3499 - val_accuracy: 0.8571
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 0.3280 - accuracy: 0.8438

3/3 [==============================] - 1s 384ms/step - loss: 0.4017 - accuracy: 0.7722 - val_loss: 0.3346 - val_accuracy: 0.8571


In [ ]:
model_FCN.evaluate(valid_ds_ECG)

1/1 [==============================] - 0s 25ms/step - loss: 0.3346 - accuracy: 0.8571


[0.3346192538738251, 0.8571428656578064]

In [ ]:

def create_BiDirectional(input_shape, num_labels):      
    model = keras.models.Sequential([
      keras.layers.Bidirectional(keras.layers.LSTM(10, return_sequences=True), input_shape= input_shape),
      keras.layers.Bidirectional(keras.layers.LSTM(10)),
      keras.layers.Dense(num_labels, activation= ('softmax'))
      ])
      
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
y_train_Pro.shape, y_train_Pro_encoded.shape, X_train_Pro.shape

((400,), (400, 6), (400, 80))

In [ ]:
model_bidir = create_BiDirectional((80,1),6)
history = model_bidir.fit(x= train_ds_Pro, validation_data= valid_ds_Pro, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 1.7308 - accuracy: 0.2906

10/10 [==============================] - 48s 4s/step - loss: 1.7308 - accuracy: 0.2906 - val_loss: 1.6998 - val_accuracy: 0.4250
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 1.6588 - accuracy: 0.4563

10/10 [==============================] - 36s 4s/step - loss: 1.6588 - accuracy: 0.4563 - val_loss: 1.6360 - val_accuracy: 0.4250
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 1.5796 - accuracy: 0.4563

10/10 [==============================] - 35s 4s/step - loss: 1.5796 - accuracy: 0.4563 - val_loss: 1.5677 - val_accuracy: 0.4250
Epoch 4/10
10/10 [==============================] - ETA: 0s - loss: 1.4908 - accuracy: 0.4563

10/10 [==============================] - 35s 4s/step - loss: 1.4908 - accuracy: 0.4563 - val_loss: 1.5143 - val_accuracy: 0.4250
Epoch 5/10
10/10 [==============================] - ETA: 0s - loss: 1.4236 - accuracy: 0.4563

...

10/10 [==============================] - 36s 4s/step - loss: 1.3920 - accuracy: 0.4563 - val_loss: 1.5030 - val_accuracy: 0.4250
Epoch 7/10
10/10 [==============================] - ETA: 0s - loss: 1.3684 - accuracy: 0.4563

10/10 [==============================] - 36s 4s/step - loss: 1.3684 - accuracy: 0.4563 - val_loss: 1.5015 - val_accuracy: 0.4250
Epoch 8/10
10/10 [==============================] - ETA: 0s - loss: 1.3557 - accuracy: 0.4563

10/10 [==============================] - 35s 4s/step - loss: 1.3557 - accuracy: 0.4563 - val_loss: 1.5034 - val_accuracy: 0.4250
Epoch 9/10
10/10 [==============================] - ETA: 0s - loss: 1.3433 - accuracy: 0.4563

10/10 [==============================] - 34s 4s/step - loss: 1.3433 - accuracy: 0.4563 - val_loss: 1.5034 - val_accuracy: 0.4250
Epoch 10/10
10/10 [==============================] - ETA: 0s - loss: 1.3321 - accuracy: 0.4563

10/10 [==============================] - 37s 4s/step - loss: 1.3321 - accuracy: 0.4563 - val_loss: 1.5004 - val_accuracy: 0.4250


In [ ]:
model_bidir.evaluate(test_ds_Pro)

7/7 [==============================] - 2s 30ms/step - loss: 1.4269 - accuracy: 0.3512


[1.4269272089004517, 0.35121950507164]

In [ ]:
model_bidir = create_BiDirectional((144,1),7)
history = model_bidir.fit(x= train_ds_Plane, validation_data= valid_ds_Plane, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 1.9600 - accuracy: 0.1429

3/3 [==============================] - 43s 16s/step - loss: 1.9600 - accuracy: 0.1429 - val_loss: 1.9419 - val_accuracy: 0.1429
Epoch 2/10
3/3 [==============================] - ETA: 0s - loss: 1.9466 - accuracy: 0.1429

3/3 [==============================] - 33s 16s/step - loss: 1.9466 - accuracy: 0.1429 - val_loss: 1.9333 - val_accuracy: 0.1429
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 1.9384 - accuracy: 0.1429

3/3 [==============================] - 32s 16s/step - loss: 1.9384 - accuracy: 0.1429 - val_loss: 1.9255 - val_accuracy: 0.1429
Epoch 4/10
3/3 [==============================] - ETA: 0s - loss: 1.9300 - accuracy: 0.1429

3/3 [==============================] - 31s 15s/step - loss: 1.9300 - accuracy: 0.1429 - val_loss: 1.9186 - val_accuracy: 0.1429
Epoch 5/10
3/3 [==============================] - ETA: 0s - loss: 1.9249 - accuracy: 0.1429

...

3/3 [==============================] - 33s 16s/step - loss: 1.9188 - accuracy: 0.1429 - val_loss: 1.9054 - val_accuracy: 0.1905
Epoch 7/10
3/3 [==============================] - ETA: 0s - loss: 1.9138 - accuracy: 0.2143

3/3 [==============================] - 31s 15s/step - loss: 1.9138 - accuracy: 0.2143 - val_loss: 1.8996 - val_accuracy: 0.2381
Epoch 8/10
3/3 [==============================] - ETA: 0s - loss: 1.9080 - accuracy: 0.1905

3/3 [==============================] - 32s 16s/step - loss: 1.9080 - accuracy: 0.1905 - val_loss: 1.8939 - val_accuracy: 0.1905
Epoch 9/10
3/3 [==============================] - ETA: 0s - loss: 1.9028 - accuracy: 0.1905

3/3 [==============================] - 32s 16s/step - loss: 1.9028 - accuracy: 0.1905 - val_loss: 1.8887 - val_accuracy: 0.1905
Epoch 10/10
3/3 [==============================] - ETA: 0s - loss: 1.8968 - accuracy: 0.1905

3/3 [==============================] - 32s 16s/step - loss: 1.8968 - accuracy: 0.1905 - val_loss: 1.8831 - val_accuracy: 0.1905


In [ ]:
model_bidir.evaluate(test_ds_Plane)

4/4 [==============================] - 2s 39ms/step - loss: 1.9621 - accuracy: 0.0952


[1.9620779752731323, 0.095238097012043]

In [ ]:
model_bidir = create_BiDirectional((96,1),2)
history = model_bidir.fit(x= train_ds_ECG, validation_data= valid_ds_ECG, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 0.6417 - accuracy: 0.6582

3/3 [==============================] - 34s 14s/step - loss: 0.6417 - accuracy: 0.6582 - val_loss: 0.6335 - val_accuracy: 0.7143
Epoch 2/10
3/3 [==============================] - ETA: 0s - loss: 0.6349 - accuracy: 0.6709

3/3 [==============================] - 27s 13s/step - loss: 0.6349 - accuracy: 0.6709 - val_loss: 0.6286 - val_accuracy: 0.7143
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 0.6272 - accuracy: 0.6962

3/3 [==============================] - 26s 13s/step - loss: 0.6272 - accuracy: 0.6962 - val_loss: 0.6249 - val_accuracy: 0.7143
Epoch 4/10
3/3 [==============================] - ETA: 0s - loss: 0.6228 - accuracy: 0.6962

3/3 [==============================] - 26s 13s/step - loss: 0.6228 - accuracy: 0.6962 - val_loss: 0.6212 - val_accuracy: 0.7143
Epoch 5/10
3/3 [==============================] - ETA: 0s - loss: 0.6194 - accuracy: 0.6962

...

3/3 [==============================] - 27s 14s/step - loss: 0.6153 - accuracy: 0.6962 - val_loss: 0.6145 - val_accuracy: 0.7143
Epoch 7/10
3/3 [==============================] - ETA: 0s - loss: 0.6110 - accuracy: 0.6962

3/3 [==============================] - 26s 13s/step - loss: 0.6110 - accuracy: 0.6962 - val_loss: 0.6113 - val_accuracy: 0.7143
Epoch 8/10
3/3 [==============================] - ETA: 0s - loss: 0.6077 - accuracy: 0.6962

3/3 [==============================] - 26s 13s/step - loss: 0.6077 - accuracy: 0.6962 - val_loss: 0.6077 - val_accuracy: 0.7143
Epoch 9/10
3/3 [==============================] - ETA: 0s - loss: 0.6038 - accuracy: 0.6962

3/3 [==============================] - 26s 13s/step - loss: 0.6038 - accuracy: 0.6962 - val_loss: 0.6033 - val_accuracy: 0.7143
Epoch 10/10
3/3 [==============================] - ETA: 0s - loss: 0.5996 - accuracy: 0.6962

3/3 [==============================] - 26s 13s/step - loss: 0.5996 - accuracy: 0.6962 - val_loss: 0.5981 - val_accuracy: 0.7143


In [ ]:
model_bidir.evaluate(test_ds_ECG)

4/4 [==============================] - 2s 21ms/step - loss: 0.6359 - accuracy: 0.6400


[0.6358933448791504, 0.6399999856948853]

In [ ]:
# 1D_CNN
def create_1D_CNN(input_shape, num_labels):
    model= tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64,kernel_size=3,activation='relu', input_shape=input_shape), 
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=64,kernel_size=3,activation='relu'), 
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_labels, activation='sigmoid')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
#input_shape = (80, 1)
model_CNN = create_1D_CNN((80, 1),6)
history = model_CNN.fit(x= train_ds_Pro, validation_data= valid_ds_Pro, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
 8/10 [=======================>......] - ETA: 0s - loss: 1.5335 - accuracy: 0.4180

10/10 [==============================] - 3s 227ms/step - loss: 1.4955 - accuracy: 0.4219 - val_loss: 1.5615 - val_accuracy: 0.4250
Epoch 2/10
 8/10 [=======================>......] - ETA: 0s - loss: 1.3702 - accuracy: 0.4648

10/10 [==============================] - 2s 192ms/step - loss: 1.3325 - accuracy: 0.4656 - val_loss: 1.5377 - val_accuracy: 0.4250
Epoch 3/10
 9/10 [==========================>...] - ETA: 0s - loss: 1.2434 - accuracy: 0.4688

10/10 [==============================] - 2s 180ms/step - loss: 1.2490 - accuracy: 0.4688 - val_loss: 1.4364 - val_accuracy: 0.6250
Epoch 4/10
 9/10 [==========================>...] - ETA: 0s - loss: 1.1565 - accuracy: 0.6910

10/10 [==============================] - 2s 262ms/step - loss: 1.1343 - accuracy: 0.7000 - val_loss: 1.3050 - val_accuracy: 0.6125
Epoch 5/10
10/10 [==============================] - ETA: 0s - loss: 0.9875 - accuracy: 0.7063

...

10/10 [==============================] - 2s 172ms/step - loss: 0.8387 - accuracy: 0.7188 - val_loss: 1.0736 - val_accuracy: 0.6750
Epoch 7/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.7428 - accuracy: 0.7535

10/10 [==============================] - 2s 188ms/step - loss: 0.7503 - accuracy: 0.7469 - val_loss: 1.0609 - val_accuracy: 0.6750
Epoch 8/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.7361 - accuracy: 0.7639

10/10 [==============================] - 4s 483ms/step - loss: 0.6989 - accuracy: 0.7719 - val_loss: 1.0485 - val_accuracy: 0.6750
Epoch 9/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.6557 - accuracy: 0.7708

10/10 [==============================] - 2s 251ms/step - loss: 0.6751 - accuracy: 0.7656 - val_loss: 1.0452 - val_accuracy: 0.6625
Epoch 10/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.6377 - accuracy: 0.7743

10/10 [==============================] - 3s 338ms/step - loss: 0.6524 - accuracy: 0.7688 - val_loss: 1.0150 - val_accuracy: 0.6875


In [ ]:
model_CNN.evaluate(test_ds_Pro)

7/7 [==============================] - 0s 7ms/step - loss: 0.8173 - accuracy: 0.7171


[0.8173232674598694, 0.7170731425285339]

In [ ]:
model_CNN = create_1D_CNN((144, 1),7)
history = model_CNN.fit(x= train_ds_Plane, validation_data= valid_ds_Plane, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 1.8987 - accuracy: 0.2381

3/3 [==============================] - 3s 774ms/step - loss: 1.8987 - accuracy: 0.2381 - val_loss: 1.7063 - val_accuracy: 0.2857
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 1.6579 - accuracy: 0.3750

3/3 [==============================] - 2s 889ms/step - loss: 1.5484 - accuracy: 0.5357 - val_loss: 1.3663 - val_accuracy: 0.4762
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 1.1945 - accuracy: 0.6429

3/3 [==============================] - 2s 991ms/step - loss: 1.1945 - accuracy: 0.6429 - val_loss: 0.9150 - val_accuracy: 0.6190
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 1.0364 - accuracy: 0.5938

3/3 [==============================] - 1s 703ms/step - loss: 0.8214 - accuracy: 0.7262 - val_loss: 0.5587 - val_accuracy: 1.0000
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5861 - accuracy: 0.9375

...

3/3 [==============================] - 2s 1s/step - loss: 0.3040 - accuracy: 0.9524 - val_loss: 0.2095 - val_accuracy: 1.0000
Epoch 7/10
1/3 [=========>....................] - ETA: 0s - loss: 0.2625 - accuracy: 0.9062

3/3 [==============================] - 1s 709ms/step - loss: 0.2046 - accuracy: 0.9405 - val_loss: 0.1336 - val_accuracy: 1.0000
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1148 - accuracy: 1.0000

3/3 [==============================] - 2s 743ms/step - loss: 0.1200 - accuracy: 0.9762 - val_loss: 0.0662 - val_accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - ETA: 0s - loss: 0.0879 - accuracy: 0.9762

3/3 [==============================] - 2s 1s/step - loss: 0.0879 - accuracy: 0.9762 - val_loss: 0.0499 - val_accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - ETA: 0s - loss: 0.0774 - accuracy: 0.9643

3/3 [==============================] - 2s 897ms/step - loss: 0.0774 - accuracy: 0.9643 - val_loss: 0.0520 - val_accuracy: 1.0000


In [ ]:
model_CNN.evaluate(test_ds_Plane)

4/4 [==============================] - 0s 9ms/step - loss: 0.1180 - accuracy: 0.9524


[0.11802361160516739, 0.9523809552192688]

In [ ]:
model_CNN = create_1D_CNN((96, 1),2)
history = model_CNN.fit(x= train_ds_ECG, validation_data= valid_ds_ECG, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 0.6528 - accuracy: 0.5190

3/3 [==============================] - 4s 1s/step - loss: 0.6528 - accuracy: 0.5190 - val_loss: 0.5370 - val_accuracy: 0.6667
Epoch 2/10
3/3 [==============================] - ETA: 0s - loss: 0.5338 - accuracy: 0.6962

3/3 [==============================] - 2s 782ms/step - loss: 0.5338 - accuracy: 0.6962 - val_loss: 0.4633 - val_accuracy: 0.7619
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4315 - accuracy: 0.7188

3/3 [==============================] - 1s 673ms/step - loss: 0.4483 - accuracy: 0.7468 - val_loss: 0.4171 - val_accuracy: 0.7619
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4494 - accuracy: 0.7812

3/3 [==============================] - 3s 1s/step - loss: 0.4605 - accuracy: 0.7848 - val_loss: 0.3659 - val_accuracy: 0.8095
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4043 - accuracy: 0.8438

...

3/3 [==============================] - 2s 781ms/step - loss: 0.3830 - accuracy: 0.7848 - val_loss: 0.3349 - val_accuracy: 0.8095
Epoch 7/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4643 - accuracy: 0.7188

3/3 [==============================] - 2s 985ms/step - loss: 0.3730 - accuracy: 0.7848 - val_loss: 0.2909 - val_accuracy: 0.8571
Epoch 8/10
3/3 [==============================] - ETA: 0s - loss: 0.3342 - accuracy: 0.8354

3/3 [==============================] - 2s 1s/step - loss: 0.3342 - accuracy: 0.8354 - val_loss: 0.2724 - val_accuracy: 0.8095
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 0.2997 - accuracy: 0.9062

3/3 [==============================] - 1s 739ms/step - loss: 0.3162 - accuracy: 0.8608 - val_loss: 0.2661 - val_accuracy: 0.9048
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 0.2972 - accuracy: 0.8750

3/3 [==============================] - 3s 1s/step - loss: 0.3033 - accuracy: 0.8608 - val_loss: 0.2519 - val_accuracy: 0.9048


In [ ]:
model_CNN.evaluate(test_ds_ECG)

4/4 [==============================] - 0s 10ms/step - loss: 0.3951 - accuracy: 0.8200


[0.3951289653778076, 0.8199999928474426]

In [ ]:
# 1D_CNN + Simple RNN

def create_1D_CNN_SimpleRNN(input_shape, num_labels):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling1D(pool_size=2),
      tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
      tf.keras.layers.MaxPooling1D(pool_size=2),
      tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()), # Flatten output of the second MaxPooling1D for each time step
      tf.keras.layers.SimpleRNN(64), # Add Simple RNN 
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(num_labels, activation='sigmoid')
  ])
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model



In [ ]:
model_CNN1D_SimpleRNN = create_1D_CNN_SimpleRNN((80,1),6)
history = model_CNN1D_SimpleRNN.fit(x= train_ds_Pro, validation_data= valid_ds_Pro, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
 9/10 [==========================>...] - ETA: 0s - loss: 1.5817 - accuracy: 0.3715

10/10 [==============================] - 8s 624ms/step - loss: 1.5424 - accuracy: 0.3938 - val_loss: 1.5240 - val_accuracy: 0.4250
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 1.3507 - accuracy: 0.4563

10/10 [==============================] - 4s 400ms/step - loss: 1.3507 - accuracy: 0.4563 - val_loss: 1.5592 - val_accuracy: 0.4250
Epoch 3/10
 9/10 [==========================>...] - ETA: 0s - loss: 1.3143 - accuracy: 0.5417

10/10 [==============================] - 4s 398ms/step - loss: 1.3151 - accuracy: 0.5312 - val_loss: 1.5252 - val_accuracy: 0.4250
Epoch 4/10
 9/10 [==========================>...] - ETA: 0s - loss: 1.2322 - accuracy: 0.5312

10/10 [==============================] - 5s 509ms/step - loss: 1.2277 - accuracy: 0.5469 - val_loss: 1.3568 - val_accuracy: 0.6375
Epoch 5/10
10/10 [==============================] - ETA: 0s - loss: 1.0341 - accuracy: 0.7094

...

10/10 [==============================] - 4s 436ms/step - loss: 0.8362 - accuracy: 0.7344 - val_loss: 1.0580 - val_accuracy: 0.6750
Epoch 7/10
10/10 [==============================] - ETA: 0s - loss: 0.7443 - accuracy: 0.7312

10/10 [==============================] - 3s 362ms/step - loss: 0.7443 - accuracy: 0.7312 - val_loss: 1.0288 - val_accuracy: 0.6750
Epoch 8/10
10/10 [==============================] - ETA: 0s - loss: 0.6945 - accuracy: 0.7344

10/10 [==============================] - 5s 533ms/step - loss: 0.6945 - accuracy: 0.7344 - val_loss: 1.0116 - val_accuracy: 0.6750
Epoch 9/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.6726 - accuracy: 0.7500

10/10 [==============================] - 4s 466ms/step - loss: 0.6767 - accuracy: 0.7469 - val_loss: 0.9547 - val_accuracy: 0.6750
Epoch 10/10
10/10 [==============================] - ETA: 0s - loss: 0.6328 - accuracy: 0.7656

10/10 [==============================] - 3s 383ms/step - loss: 0.6328 - accuracy: 0.7656 - val_loss: 0.9505 - val_accuracy: 0.7000


In [ ]:
model_CNN1D_SimpleRNN.evaluate(test_ds_Pro)

7/7 [==============================] - 0s 8ms/step - loss: 0.7980 - accuracy: 0.8000


[0.7979767322540283, 0.800000011920929]

In [ ]:
model_CNN1D_SimpleRNN = create_1D_CNN_SimpleRNN((144,1),7)
model_CNN1D_SimpleRNN.fit(x= train_ds_Plane, validation_data =valid_ds_Plane, epochs=10, batch_size=32, callbacks=[early_stopping, checkpoint])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 1.8789 - accuracy: 0.2976

3/3 [==============================] - 7s 2s/step - loss: 1.8789 - accuracy: 0.2976 - val_loss: 1.7341 - val_accuracy: 0.2857
Epoch 2/10
2/3 [===================>..........] - ETA: 0s - loss: 1.6823 - accuracy: 0.4062

3/3 [==============================] - 5s 2s/step - loss: 1.6701 - accuracy: 0.4167 - val_loss: 1.5931 - val_accuracy: 0.6667
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 1.5242 - accuracy: 0.5119

3/3 [==============================] - 7s 4s/step - loss: 1.5242 - accuracy: 0.5119 - val_loss: 1.4616 - val_accuracy: 0.4286
Epoch 4/10
3/3 [==============================] - ETA: 0s - loss: 1.3656 - accuracy: 0.5476

3/3 [==============================] - 9s 4s/step - loss: 1.3656 - accuracy: 0.5476 - val_loss: 1.3197 - val_accuracy: 0.6190
Epoch 5/10
3/3 [==============================] - ETA: 0s - loss: 1.2051 - accuracy: 0.7024

...

3/3 [==============================] - 3s 2s/step - loss: 1.0353 - accuracy: 0.9048 - val_loss: 0.9665 - val_accuracy: 0.9524
Epoch 7/10
3/3 [==============================] - ETA: 0s - loss: 0.8855 - accuracy: 0.8929

3/3 [==============================] - 3s 2s/step - loss: 0.8855 - accuracy: 0.8929 - val_loss: 0.8192 - val_accuracy: 1.0000
Epoch 8/10
3/3 [==============================] - ETA: 0s - loss: 0.7453 - accuracy: 0.8929

3/3 [==============================] - 4s 2s/step - loss: 0.7453 - accuracy: 0.8929 - val_loss: 0.6738 - val_accuracy: 0.9524
Epoch 9/10
3/3 [==============================] - ETA: 0s - loss: 0.6301 - accuracy: 0.9167

3/3 [==============================] - 3s 2s/step - loss: 0.6301 - accuracy: 0.9167 - val_loss: 0.5392 - val_accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - ETA: 0s - loss: 0.5297 - accuracy: 0.9048

3/3 [==============================] - 5s 2s/step - loss: 0.5297 - accuracy: 0.9048 - val_loss: 0.4324 - val_accuracy: 1.0000


In [ ]:
model_CNN1D_SimpleRNN.evaluate(test_ds_Plane)

4/4 [==============================] - 1s 11ms/step - loss: 0.5498 - accuracy: 0.9429


[0.5498329401016235, 0.9428571462631226]

In [ ]:
model_CNN1D_SimpleRNN = create_1D_CNN_SimpleRNN((96,1),2)
model_CNN1D_SimpleRNN.fit(x=train_ds_ECG, validation_data= valid_ds_ECG, epochs=10, batch_size=32,callbacks=[early_stopping, checkpoint] )

Epoch 1/10
1/3 [=========>....................] - ETA: 4s - loss: 0.6949 - accuracy: 0.3125

3/3 [==============================] - 7s 2s/step - loss: 0.6429 - accuracy: 0.5316 - val_loss: 0.5126 - val_accuracy: 0.7619
Epoch 2/10
3/3 [==============================] - ETA: 0s - loss: 0.5076 - accuracy: 0.7595

3/3 [==============================] - 3s 2s/step - loss: 0.5076 - accuracy: 0.7595 - val_loss: 0.4495 - val_accuracy: 0.8095
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5274 - accuracy: 0.7188

3/3 [==============================] - 4s 2s/step - loss: 0.4431 - accuracy: 0.7848 - val_loss: 0.3709 - val_accuracy: 0.7619
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 0.3388 - accuracy: 0.7812

3/3 [==============================] - 4s 2s/step - loss: 0.4176 - accuracy: 0.7595 - val_loss: 0.3287 - val_accuracy: 0.8095
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 0.4179 - accuracy: 0.7812

...

3/3 [==============================] - 3s 2s/step - loss: 0.3640 - accuracy: 0.8734 - val_loss: 0.3363 - val_accuracy: 0.8095
Epoch 7/10
3/3 [==============================] - ETA: 0s - loss: 0.3363 - accuracy: 0.8734

3/3 [==============================] - 5s 3s/step - loss: 0.3363 - accuracy: 0.8734 - val_loss: 0.3253 - val_accuracy: 0.8571
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1687 - accuracy: 1.0000

3/3 [==============================] - 3s 2s/step - loss: 0.3108 - accuracy: 0.9114 - val_loss: 0.2882 - val_accuracy: 0.9048
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 0.3643 - accuracy: 0.8750

3/3 [==============================] - 4s 2s/step - loss: 0.2967 - accuracy: 0.9241 - val_loss: 0.2634 - val_accuracy: 0.9048
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 0.2398 - accuracy: 0.9688

3/3 [==============================] - 3s 2s/step - loss: 0.2843 - accuracy: 0.9367 - val_loss: 0.2645 - val_accuracy: 0.9048


In [ ]:
model_CNN1D_SimpleRNN.evaluate(X_test_ECG, y_test_ECG_encoded)

4/4 [==============================] - 1s 13ms/step - loss: 0.4370 - accuracy: 0.8000


[0.43700870871543884, 0.800000011920929]

### Task 2: Time series classification using deep learning 2

There has been several neural network models dedicated to time series classification. Besides our own models that you developed in Lab 4, now we will develop such dedicated models by referring to some papers, and test if they indeed perform better than our rough models. There are two famous papers as follows:
 - [Convolutional neural networks for time series classification (2017)](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7870510)
 - [Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2017)](https://arxiv.org/abs/1611.06455)

First paper's idea is already implemented in sktime, with the name `CNNClassifier`. Second paper has three models and those are easy to develop using tensorflow. Now the task is to develop two models (MLP and FCN) in the second paper and test it together with `CNNClassifier`.

Using the same datasets, and test sktime's `CNNClassifier` and MLP and FCN models developed. Reporting test scores of three models (`CNNClassifier`, MLP, and FCN) on three datasets you chose. It would be nine scores in total. 


In [ ]:
#implement CNN Classifier
from sktime.classification.deep_learning.cnn import CNNClassifier

/usr/local/lib/python3.9/dist-packages/sktime/utils/validation/_dependencies.py:143: UserWarning: No module named 'keras_self_attention'. 'keras-self-attention' is a soft dependency and not included in the base sktime installation. Please run: `pip install keras-self-attention` to install the keras-self-attention package. To install all soft dependencies, run: `pip install sktime[all_extras]`
  warnings.warn(msg)


In [ ]:

#import datasets again, the purpose is not using tf.Data
X_train_Pro_CNN, y_train_Pro_CNN = load_UCR_UEA_dataset(name='ProximalPhalanxTW', split='train', return_type="numpy2D")
X_test_Pro_CNN , y_test_Pro_CNN = load_UCR_UEA_dataset(name='ProximalPhalanxTW', split='test', return_type="numpy2D")

In [ ]:
CNN_classifier = CNNClassifier(n_epochs= 10)
CNN_classifier.fit(X_train_Pro_CNN, y_train_Pro_CNN)
accuracy = CNN_classifier.score(X_test_Pro_CNN , y_test_Pro_CNN)
print('Test accuracy:', accuracy)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


13/13 [==============================] - 0s 3ms/step
Test accuracy: 0.35121951219512193


In [ ]:
X_train_Plane_CNN, y_train_Plane_CNN = load_UCR_UEA_dataset(name='Plane', split='train', return_type="numpy2D")
X_test_Plane_CNN , y_test_Plane_CNN = load_UCR_UEA_dataset(name='Plane', split='test', return_type="numpy2D")

In [ ]:
CNN_classifier = CNNClassifier(n_epochs= 10)
CNN_classifier.fit(X_train_Plane_CNN, y_train_Plane_CNN)
accuracy = CNN_classifier.score(X_test_Plane_CNN , y_test_Plane_CNN)
print('Test accuracy:', accuracy)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


7/7 [==============================] - 0s 6ms/step
Test accuracy: 0.5904761904761905


In [ ]:
X_train_ECG_CNN, y_train_ECG_CNN = load_UCR_UEA_dataset(name='Plane', split='train', return_type="numpy2D")
X_test_ECG_CNN , y_test_ECG_CNN = load_UCR_UEA_dataset(name='Plane', split='test', return_type="numpy2D")

In [ ]:
CNN_classifier = CNNClassifier(n_epochs= 10)
CNN_classifier.fit(X_train_ECG_CNN, y_train_ECG_CNN)
accuracy = CNN_classifier.score(X_test_ECG_CNN , y_test_ECG_CNN)
print('Test accuracy:', accuracy)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


7/7 [==============================] - 0s 5ms/step
Test accuracy: 0.6571428571428571


In [ ]:
#implement MLP

def creat_MLP(input_shape, num_label):
  model= keras.models.Sequential([
      keras.layers.Input(shape = input_shape),
      keras.layers.Dropout(0.1, input_shape=[input_shape]),
      keras.layers.Dense(500, activation='relu'),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(500, activation='relu'),
      keras.layers.Dropout(0.3),
      keras.layers.Dense(500, activation='relu'),
      keras.layers.Dense(num_label, activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model_MLP = creat_MLP(80, 6)
history = model_MLP.fit(X_train_Pro, y_train_Pro_encoded, validation_split=0.2, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 1.5874 - accuracy: 0.3969

10/10 [==============================] - 3s 225ms/step - loss: 1.5874 - accuracy: 0.3969 - val_loss: 1.1939 - val_accuracy: 0.4875
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 1.2539 - accuracy: 0.5406

10/10 [==============================] - 2s 192ms/step - loss: 1.2539 - accuracy: 0.5406 - val_loss: 0.8888 - val_accuracy: 0.7875
Epoch 3/10
 9/10 [==========================>...] - ETA: 0s - loss: 1.0413 - accuracy: 0.6181

10/10 [==============================] - 3s 276ms/step - loss: 1.0143 - accuracy: 0.6344 - val_loss: 0.6085 - val_accuracy: 0.7875
Epoch 4/10
 8/10 [=======================>......] - ETA: 0s - loss: 0.7996 - accuracy: 0.7344

10/10 [==============================] - 4s 393ms/step - loss: 0.8328 - accuracy: 0.7125 - val_loss: 0.7478 - val_accuracy: 0.6125
Epoch 5/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.8278 - accuracy: 0.7083

...

10/10 [==============================] - 2s 199ms/step - loss: 0.7607 - accuracy: 0.7312 - val_loss: 0.4875 - val_accuracy: 0.8500
Epoch 7/10
10/10 [==============================] - ETA: 0s - loss: 0.7239 - accuracy: 0.7500

10/10 [==============================] - 2s 217ms/step - loss: 0.7239 - accuracy: 0.7500 - val_loss: 0.4924 - val_accuracy: 0.8875
Epoch 8/10
10/10 [==============================] - ETA: 0s - loss: 0.7250 - accuracy: 0.7344

10/10 [==============================] - 2s 267ms/step - loss: 0.7250 - accuracy: 0.7344 - val_loss: 0.4658 - val_accuracy: 0.8875
Epoch 9/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.6868 - accuracy: 0.7500

10/10 [==============================] - 2s 233ms/step - loss: 0.6860 - accuracy: 0.7469 - val_loss: 0.4522 - val_accuracy: 0.8250
Epoch 10/10
 7/10 [====================>.........] - ETA: 0s - loss: 0.7392 - accuracy: 0.7098

10/10 [==============================] - 2s 195ms/step - loss: 0.7345 - accuracy: 0.7125 - val_loss: 0.5939 - val_accuracy: 0.7875


In [ ]:
model_MLP.evaluate(test_ds_Pro)

7/7 [==============================] - 0s 6ms/step - loss: 0.7891 - accuracy: 0.7268


[0.7891157865524292, 0.7268292903900146]

In [ ]:
model_MLP = creat_MLP(144, 7)
history = model_MLP.fit(X_train_Plane, y_train_Plane_encoded, validation_split=0.2, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
1/3 [=========>....................] - ETA: 2s - loss: 2.0326 - accuracy: 0.0938

3/3 [==============================] - 4s 2s/step - loss: 1.7843 - accuracy: 0.3214 - val_loss: 1.3720 - val_accuracy: 0.5238
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 1.1831 - accuracy: 0.5938

3/3 [==============================] - 2s 813ms/step - loss: 1.0281 - accuracy: 0.6190 - val_loss: 0.6940 - val_accuracy: 0.7143
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 0.6789 - accuracy: 0.8438

3/3 [==============================] - 2s 801ms/step - loss: 0.5511 - accuracy: 0.8571 - val_loss: 0.5902 - val_accuracy: 0.7143
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 0.3930 - accuracy: 0.8750

3/3 [==============================] - 2s 785ms/step - loss: 0.4308 - accuracy: 0.8333 - val_loss: 0.3825 - val_accuracy: 0.9048
Epoch 5/10
3/3 [==============================] - ETA: 0s - loss: 0.2909 - accuracy: 0.9048

...

3/3 [==============================] - 2s 989ms/step - loss: 0.1877 - accuracy: 0.9643 - val_loss: 0.3063 - val_accuracy: 0.9524
Epoch 7/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1869 - accuracy: 0.9062

3/3 [==============================] - 2s 767ms/step - loss: 0.1506 - accuracy: 0.9405 - val_loss: 0.2719 - val_accuracy: 0.9524
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 0.0824 - accuracy: 1.0000

3/3 [==============================] - 2s 825ms/step - loss: 0.0944 - accuracy: 0.9762 - val_loss: 0.2026 - val_accuracy: 0.9524
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1778 - accuracy: 0.9062

3/3 [==============================] - 2s 758ms/step - loss: 0.1173 - accuracy: 0.9524 - val_loss: 0.3707 - val_accuracy: 0.8571
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1757 - accuracy: 0.9375

3/3 [==============================] - 3s 1s/step - loss: 0.1090 - accuracy: 0.9643 - val_loss: 0.2807 - val_accuracy: 0.9524


In [ ]:
model_MLP.evaluate(X_test_Plane, y_test_Plane_encoded)

4/4 [==============================] - 0s 8ms/step - loss: 0.1261 - accuracy: 0.9714


[0.12611117959022522, 0.9714285731315613]

In [ ]:
model_MLP = creat_MLP(96, 2)
history = model_MLP.fit(X_train_ECG, y_train_ECG_encoded, validation_split=0.2, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
1/3 [=========>....................] - ETA: 2s - loss: 0.7841 - accuracy: 0.4375

3/3 [==============================] - 3s 892ms/step - loss: 0.9120 - accuracy: 0.5375 - val_loss: 0.3571 - val_accuracy: 0.7500
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 0.6029 - accuracy: 0.6562

3/3 [==============================] - 2s 813ms/step - loss: 0.5187 - accuracy: 0.7625 - val_loss: 0.3091 - val_accuracy: 0.9000
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5103 - accuracy: 0.8125

3/3 [==============================] - 2s 779ms/step - loss: 0.4329 - accuracy: 0.8250 - val_loss: 0.2471 - val_accuracy: 0.9000
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 0.2867 - accuracy: 0.8750

3/3 [==============================] - 1s 720ms/step - loss: 0.3352 - accuracy: 0.8375 - val_loss: 0.2817 - val_accuracy: 0.9000
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 0.2284 - accuracy: 0.9062

...

3/3 [==============================] - 2s 913ms/step - loss: 0.2543 - accuracy: 0.9000 - val_loss: 0.2842 - val_accuracy: 0.8500
Epoch 7/10
3/3 [==============================] - ETA: 0s - loss: 0.2577 - accuracy: 0.8875

3/3 [==============================] - 2s 1s/step - loss: 0.2577 - accuracy: 0.8875 - val_loss: 0.3302 - val_accuracy: 0.8500
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1735 - accuracy: 0.9375

3/3 [==============================] - 2s 803ms/step - loss: 0.2662 - accuracy: 0.8875 - val_loss: 0.4489 - val_accuracy: 0.8500
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1850 - accuracy: 0.9062

3/3 [==============================] - 3s 1s/step - loss: 0.1962 - accuracy: 0.9000 - val_loss: 0.3497 - val_accuracy: 0.9000
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 0.1979 - accuracy: 0.8750

3/3 [==============================] - 2s 784ms/step - loss: 0.1451 - accuracy: 0.9125 - val_loss: 0.2941 - val_accuracy: 0.9000


In [ ]:
model_MLP.evaluate(X_test_ECG, y_test_ECG_encoded)

4/4 [==============================] - 0s 8ms/step - loss: 0.4015 - accuracy: 0.8600


[0.40149056911468506, 0.8600000143051147]

In [ ]:
#Fully Convolutional Network

def create_FConN(input_shape, num_labels):
  model = keras.models.Sequential([
      keras.layers.Input(shape = input_shape),
      keras.layers.Conv1D(filters= 128, kernel_size= 3, activation='relu', input_shape=input_shape),
      keras.layers.BatchNormalization(),
      keras.layers.Conv1D(filters= 256, kernel_size=3, activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.Conv1D(filters= 128, kernel_size=3, activation='relu'),
      keras.layers.BatchNormalization(),
      keras.layers.GlobalAveragePooling1D(),
      keras.layers.Dense(num_labels,activation='softmax')
  ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model_FConN = create_FConN((80,1), 6)
history = model_FConN.fit(X_train_Pro, y_train_Pro_encoded, validation_split=0.2, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 1.2418 - accuracy: 0.5813

10/10 [==============================] - 8s 532ms/step - loss: 1.2418 - accuracy: 0.5813 - val_loss: 1.7337 - val_accuracy: 0.3000
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 0.8095 - accuracy: 0.7594

10/10 [==============================] - 4s 388ms/step - loss: 0.8095 - accuracy: 0.7594 - val_loss: 1.6568 - val_accuracy: 0.3000
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 0.7467 - accuracy: 0.7531

10/10 [==============================] - 4s 398ms/step - loss: 0.7467 - accuracy: 0.7531 - val_loss: 1.5856 - val_accuracy: 0.3000
Epoch 4/10
10/10 [==============================] - ETA: 0s - loss: 0.6854 - accuracy: 0.7844

10/10 [==============================] - 6s 654ms/step - loss: 0.6854 - accuracy: 0.7844 - val_loss: 1.5133 - val_accuracy: 0.3000
Epoch 5/10
10/10 [==============================] - ETA: 0s - loss: 0.6993 - accuracy: 0.7563

...

10/10 [==============================] - 4s 406ms/step - loss: 0.6381 - accuracy: 0.7688 - val_loss: 1.5171 - val_accuracy: 0.3000
Epoch 7/10
10/10 [==============================] - ETA: 0s - loss: 0.6341 - accuracy: 0.7656

10/10 [==============================] - 5s 522ms/step - loss: 0.6341 - accuracy: 0.7656 - val_loss: 1.5744 - val_accuracy: 0.3000
Epoch 8/10
10/10 [==============================] - ETA: 0s - loss: 0.5800 - accuracy: 0.7781

10/10 [==============================] - 4s 394ms/step - loss: 0.5800 - accuracy: 0.7781 - val_loss: 1.6925 - val_accuracy: 0.3000
Epoch 9/10
10/10 [==============================] - ETA: 0s - loss: 0.5697 - accuracy: 0.7781

10/10 [==============================] - 4s 436ms/step - loss: 0.5697 - accuracy: 0.7781 - val_loss: 1.9009 - val_accuracy: 0.3000
Epoch 10/10
10/10 [==============================] - ETA: 0s - loss: 0.5620 - accuracy: 0.7781

10/10 [==============================] - 5s 503ms/step - loss: 0.5620 - accuracy: 0.7781 - val_loss: 2.0916 - val_accuracy: 0.3000


In [ ]:
model_FConN.evaluate(X_test_Pro, y_test_Pro_encoded)

7/7 [==============================] - 0s 28ms/step - loss: 2.2987 - accuracy: 0.3268


[2.2987422943115234, 0.3268292546272278]

In [ ]:
model_FConN = create_FConN((144,1), 7)
history = model_FConN.fit(X_train_Plane, y_train_Plane_encoded, validation_split=0.2, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 1.7466 - accuracy: 0.4762

3/3 [==============================] - 7s 2s/step - loss: 1.7466 - accuracy: 0.4762 - val_loss: 1.9639 - val_accuracy: 0.0476
Epoch 2/10
3/3 [==============================] - ETA: 0s - loss: 1.2901 - accuracy: 0.9286

3/3 [==============================] - 3s 1s/step - loss: 1.2901 - accuracy: 0.9286 - val_loss: 1.9690 - val_accuracy: 0.0476
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 1.0854 - accuracy: 0.9643

3/3 [==============================] - 4s 2s/step - loss: 1.0854 - accuracy: 0.9643 - val_loss: 1.9710 - val_accuracy: 0.0476
Epoch 4/10
3/3 [==============================] - ETA: 0s - loss: 0.9357 - accuracy: 0.9762

3/3 [==============================] - 3s 1s/step - loss: 0.9357 - accuracy: 0.9762 - val_loss: 1.9663 - val_accuracy: 0.0476
Epoch 5/10
3/3 [==============================] - ETA: 0s - loss: 0.8140 - accuracy: 0.9881

...

3/3 [==============================] - 5s 2s/step - loss: 0.6923 - accuracy: 0.9881 - val_loss: 1.9491 - val_accuracy: 0.0000e+00
Epoch 7/10
3/3 [==============================] - ETA: 0s - loss: 0.5750 - accuracy: 0.9881

3/3 [==============================] - 4s 2s/step - loss: 0.5750 - accuracy: 0.9881 - val_loss: 1.9407 - val_accuracy: 0.1429
Epoch 8/10
3/3 [==============================] - ETA: 0s - loss: 0.5232 - accuracy: 0.9881

3/3 [==============================] - 3s 1s/step - loss: 0.5232 - accuracy: 0.9881 - val_loss: 1.9364 - val_accuracy: 0.1429
Epoch 9/10
3/3 [==============================] - ETA: 0s - loss: 0.4095 - accuracy: 1.0000

3/3 [==============================] - 3s 1s/step - loss: 0.4095 - accuracy: 1.0000 - val_loss: 1.9369 - val_accuracy: 0.1429
Epoch 10/10
3/3 [==============================] - ETA: 0s - loss: 0.3680 - accuracy: 1.0000

3/3 [==============================] - 4s 2s/step - loss: 0.3680 - accuracy: 1.0000 - val_loss: 1.9430 - val_accuracy: 0.1429


In [ ]:
model_FConN.evaluate(X_test_Plane, y_test_Plane_encoded)

4/4 [==============================] - 0s 37ms/step - loss: 2.0543 - accuracy: 0.1143


[2.0542726516723633, 0.11428571492433548]

In [ ]:
model_FConN = create_FConN((96,1), 2)
history = model_FConN.fit(X_train_ECG, y_train_ECG_encoded, validation_split=0.2, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 0.6835 - accuracy: 0.6125

3/3 [==============================] - 8s 2s/step - loss: 0.6835 - accuracy: 0.6125 - val_loss: 0.7068 - val_accuracy: 0.2500
Epoch 2/10
3/3 [==============================] - ETA: 0s - loss: 0.5268 - accuracy: 0.7625

3/3 [==============================] - 3s 1s/step - loss: 0.5268 - accuracy: 0.7625 - val_loss: 0.6751 - val_accuracy: 0.8500
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 0.5211 - accuracy: 0.7375

3/3 [==============================] - 4s 2s/step - loss: 0.5211 - accuracy: 0.7375 - val_loss: 0.6452 - val_accuracy: 0.7500
Epoch 4/10
3/3 [==============================] - ETA: 0s - loss: 0.4821 - accuracy: 0.8375

3/3 [==============================] - 3s 1s/step - loss: 0.4821 - accuracy: 0.8375 - val_loss: 0.6246 - val_accuracy: 0.7500
Epoch 5/10
3/3 [==============================] - ETA: 0s - loss: 0.4491 - accuracy: 0.8125

...

3/3 [==============================] - 3s 1s/step - loss: 0.4284 - accuracy: 0.8000 - val_loss: 0.5910 - val_accuracy: 0.7500
Epoch 7/10
3/3 [==============================] - ETA: 0s - loss: 0.4018 - accuracy: 0.8625

3/3 [==============================] - 5s 2s/step - loss: 0.4018 - accuracy: 0.8625 - val_loss: 0.5776 - val_accuracy: 0.7500
Epoch 8/10
3/3 [==============================] - ETA: 0s - loss: 0.3871 - accuracy: 0.8625

3/3 [==============================] - 3s 1s/step - loss: 0.3871 - accuracy: 0.8625 - val_loss: 0.5672 - val_accuracy: 0.7500
Epoch 9/10
3/3 [==============================] - ETA: 0s - loss: 0.3705 - accuracy: 0.8500

3/3 [==============================] - 3s 1s/step - loss: 0.3705 - accuracy: 0.8500 - val_loss: 0.5585 - val_accuracy: 0.7500
Epoch 10/10
3/3 [==============================] - ETA: 0s - loss: 0.3539 - accuracy: 0.9000

3/3 [==============================] - 3s 1s/step - loss: 0.3539 - accuracy: 0.9000 - val_loss: 0.5550 - val_accuracy: 0.7500


In [ ]:
model_FConN.evaluate(X_test_ECG, y_test_ECG_encoded)

4/4 [==============================] - 0s 26ms/step - loss: 0.6554 - accuracy: 0.6400


[0.6553900241851807, 0.6399999856948853]

### Task 3: Time series classification using deep learning 3

Next, WE try to further improve your model by selecting one of the following ideas (or you can suggest your own ideas that are not trivial):
- Use residual connection to your model (= implement **ResNet** in [this](https://arxiv.org/abs/1611.06455) paper). 
- Add transformer/attention layer to your model (you can choose any model from Tasks 1 and 2).
- Use RNN and CNN networks separately, create two to three layers individually, and concatenate them. This means that until the third (or second) layer, you have two different networks handling the same dataset, and after that, you concatenate the output and finish with any FCN layer. Check [this post](https://stackoverflow.com/questions/59168306/how-to-combine-lstm-and-cnn-in-timeseries-classification) to get inspired.
- Apply any sktime's transformer (not attention transformer) first to the dataset and run any deep learning model you already developed in Tasks 2 and 3. In this case, you need to choose at least two transformers and apply them together.
- Train the model on multiple similar datasets and test it on one specific test set. Check if the model can be improved if it is trained on multiple datasets. However, for this, you may need to choose the similar datasets based on their classification (UCR repository has a specific dataset type such as **AUDIO** or **MOTION**).

Choosing one model from the models you have developed in Tasks 1 and 2. Select one idea, implementing it, and checking if you can improve the performance. Note that we do not need to prove that the accuracy scores increase but must explain your trials. Report test scores on three datasets you chose.

In [ ]:
# #fully connected network
def create_FCN(input_shape, num_labels):
    model = keras.models.Sequential([
    keras.layers.Dense(300, input_shape=[input_shape], activation='relu', name='dense1'),
    keras.layers.Dense(100, activation='relu', name='dense2'),
    keras.layers.Dense(num_labels, activation='sigmoid', name='output')
])
    
    model.compile(loss="categorical_crossentropy", 
                  optimizer="sgd", 
                  metrics=["accuracy"])
    return model


In [ ]:
from tensorflow.keras import layers

def create_FCN_attention(input_shape, num_labels):
    # Define input layers
    inputs = layers.Input(shape=[input_shape], name='input')
    
    # Fully connected layers
    x = layers.Dense(300, activation='relu', name='dense1')(inputs)
    x = layers.Dense(100, activation='relu', name='dense2')(x)

    # Attention layer
    query = layers.Dense(64, activation='relu', name='query')(x)
    value = layers.Dense(64, activation='relu', name='value')(x)
    attention = layers.Attention()([query, value])

    # Output layer
    outputs = layers.Dense(num_labels, activation='sigmoid', name='output')(attention)
    
    # Create model
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile model
    model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    
    return model


In [ ]:
model_FCN_att = create_FCN_attention(80,6)
history = model_FCN_att.fit(x= train_ds_Pro, validation_data= valid_ds_Pro, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
 1/10 [==>...........................] - ETA: 8s - loss: 1.7948 - accuracy: 0.0938

10/10 [==============================] - 2s 138ms/step - loss: 1.6555 - accuracy: 0.4031 - val_loss: 1.4688 - val_accuracy: 0.4875
Epoch 2/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.5751 - accuracy: 0.3750

10/10 [==============================] - 1s 145ms/step - loss: 1.5037 - accuracy: 0.4406 - val_loss: 1.4146 - val_accuracy: 0.4875
Epoch 3/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.5177 - accuracy: 0.4375

10/10 [==============================] - 1s 108ms/step - loss: 1.4552 - accuracy: 0.4406 - val_loss: 1.4145 - val_accuracy: 0.4875
Epoch 4/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.4454 - accuracy: 0.5312

10/10 [==============================] - 1s 110ms/step - loss: 1.4332 - accuracy: 0.4406 - val_loss: 1.4172 - val_accuracy: 0.4875
Epoch 5/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.3960 - accuracy: 0.4375

...

10/10 [==============================] - 1s 165ms/step - loss: 1.4155 - accuracy: 0.4406 - val_loss: 1.4266 - val_accuracy: 0.4875
Epoch 7/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.5660 - accuracy: 0.4062

10/10 [==============================] - 1s 158ms/step - loss: 1.4157 - accuracy: 0.4406 - val_loss: 1.4149 - val_accuracy: 0.4875
Epoch 8/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.2981 - accuracy: 0.5000

10/10 [==============================] - 1s 153ms/step - loss: 1.4134 - accuracy: 0.4406 - val_loss: 1.4192 - val_accuracy: 0.4875
Epoch 9/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.4687 - accuracy: 0.4375

10/10 [==============================] - 1s 108ms/step - loss: 1.4125 - accuracy: 0.4406 - val_loss: 1.4373 - val_accuracy: 0.4875
Epoch 10/10
 1/10 [==>...........................] - ETA: 0s - loss: 1.2839 - accuracy: 0.4375

10/10 [==============================] - 1s 108ms/step - loss: 1.4104 - accuracy: 0.4406 - val_loss: 1.4145 - val_accuracy: 0.4875


In [ ]:
model_FCN_att.evaluate(test_ds_Pro)

7/7 [==============================] - 0s 3ms/step - loss: 1.4786 - accuracy: 0.3512


[1.4785864353179932, 0.35121950507164]

In [ ]:
model_FCN_att = create_FCN_attention(144,7)
history = model_FCN_att.fit(x= train_ds_Plane, validation_data= valid_ds_Plane, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
1/3 [=========>....................] - ETA: 1s - loss: 2.0428 - accuracy: 0.2188

3/3 [==============================] - 2s 561ms/step - loss: 2.0068 - accuracy: 0.1905 - val_loss: 1.9411 - val_accuracy: 0.1905
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 1.9689 - accuracy: 0.1562

3/3 [==============================] - 1s 507ms/step - loss: 1.9532 - accuracy: 0.1905 - val_loss: 1.9390 - val_accuracy: 0.1905
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 1.9134 - accuracy: 0.2500

3/3 [==============================] - 1s 485ms/step - loss: 1.9295 - accuracy: 0.1905 - val_loss: 1.9288 - val_accuracy: 0.1905
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 1.9096 - accuracy: 0.1875

3/3 [==============================] - 1s 677ms/step - loss: 1.9143 - accuracy: 0.1905 - val_loss: 1.9127 - val_accuracy: 0.1905
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 1.9468 - accuracy: 0.1250

...

3/3 [==============================] - 1s 484ms/step - loss: 1.8813 - accuracy: 0.1905 - val_loss: 1.8974 - val_accuracy: 0.1905
Epoch 7/10
1/3 [=========>....................] - ETA: 0s - loss: 1.8542 - accuracy: 0.1875

3/3 [==============================] - 1s 480ms/step - loss: 1.8621 - accuracy: 0.1905 - val_loss: 1.8693 - val_accuracy: 0.1905
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 1.8569 - accuracy: 0.1250

3/3 [==============================] - 1s 502ms/step - loss: 1.8299 - accuracy: 0.1905 - val_loss: 1.8464 - val_accuracy: 0.1905
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 1.7748 - accuracy: 0.2188

3/3 [==============================] - 1s 478ms/step - loss: 1.7804 - accuracy: 0.2500 - val_loss: 1.8124 - val_accuracy: 0.1905
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 1.7279 - accuracy: 0.3750

3/3 [==============================] - 1s 483ms/step - loss: 1.7202 - accuracy: 0.3214 - val_loss: 1.7534 - val_accuracy: 0.3333


In [ ]:
model_FCN_att.evaluate(test_ds_Plane)

4/4 [==============================] - 0s 4ms/step - loss: 1.7616 - accuracy: 0.2952


[1.7616060972213745, 0.29523810744285583]

In [ ]:
model_FCN_att = create_FCN_attention(96,2)
history = model_FCN_att.fit(x= train_ds_ECG, validation_data= valid_ds_ECG, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
1/3 [=========>....................] - ETA: 2s - loss: 0.6588 - accuracy: 0.6562

3/3 [==============================] - 2s 583ms/step - loss: 0.6453 - accuracy: 0.6962 - val_loss: 0.6341 - val_accuracy: 0.6667
Epoch 2/10
1/3 [=========>....................] - ETA: 0s - loss: 0.6612 - accuracy: 0.6250

3/3 [==============================] - 1s 484ms/step - loss: 0.6202 - accuracy: 0.6962 - val_loss: 0.6362 - val_accuracy: 0.6667
Epoch 3/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5940 - accuracy: 0.7188

3/3 [==============================] - 1s 487ms/step - loss: 0.6104 - accuracy: 0.6962 - val_loss: 0.6380 - val_accuracy: 0.6667
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5905 - accuracy: 0.7188

3/3 [==============================] - 1s 490ms/step - loss: 0.6087 - accuracy: 0.6962 - val_loss: 0.6358 - val_accuracy: 0.6667
Epoch 5/10
1/3 [=========>....................] - ETA: 0s - loss: 0.6628 - accuracy: 0.6250

...

3/3 [==============================] - 1s 475ms/step - loss: 0.6069 - accuracy: 0.6962 - val_loss: 0.6304 - val_accuracy: 0.6667
Epoch 7/10
1/3 [=========>....................] - ETA: 0s - loss: 0.6138 - accuracy: 0.6875

3/3 [==============================] - 1s 497ms/step - loss: 0.6031 - accuracy: 0.6962 - val_loss: 0.6316 - val_accuracy: 0.6667
Epoch 8/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5823 - accuracy: 0.7188

3/3 [==============================] - 1s 744ms/step - loss: 0.5908 - accuracy: 0.6962 - val_loss: 0.6371 - val_accuracy: 0.6667
Epoch 9/10
1/3 [=========>....................] - ETA: 0s - loss: 0.5412 - accuracy: 0.7500

3/3 [==============================] - 1s 688ms/step - loss: 0.5972 - accuracy: 0.6962 - val_loss: 0.6240 - val_accuracy: 0.6667
Epoch 10/10
1/3 [=========>....................] - ETA: 0s - loss: 0.6088 - accuracy: 0.6875

3/3 [==============================] - 1s 584ms/step - loss: 0.5895 - accuracy: 0.6962 - val_loss: 0.6192 - val_accuracy: 0.6667


In [ ]:
model_FCN_att.evaluate(test_ds_ECG)

4/4 [==============================] - 0s 4ms/step - loss: 0.6323 - accuracy: 0.6400


[0.6322964429855347, 0.6399999856948853]

### Task 4: Time series classification using sktime

As we saw above, we can use RandomizedSearch to find optimal parameter options. However, it does not support scikit-learn's classifiers such as DecisionTree or RandomForest. However, as we tried, it is also possible to use the output of the sktime transformer (e.g., catch22) to train scikit-learn models such as RandomForest. Sktime supports this with `SklearnClassifierPipeline` to put a  scikit-learn classifier and sktime's transformer together and you need to implement it.

Picking one classifier from scikit-learn (that can be anything! e.g., Decision Tree or Logistic Regressor) and two transformers from sktime and create `SklearnClassifierPipeline.` As we tried in this lab, that can be **Rocket with RandomForest** or **Catch22 with DecisionTree**. Pick one parameter from each module (in total, three, one from the classifier and two from two transformers) and run a randomized search on the pipeline and report the test score of the best model found by the randomized Search. Comparing best score to the score from the same model with the default setting.

Task 4 involves a time-consuming process, so choosing **one dataset** to perform the task above. Also, note that we do not need to perform better by conducting a randomized search for this task (but still good to try!).

In [ ]:
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.catch22 import Catch22
from sktime.transformations.series.exponent import ExponentTransformer
from sktime.classification.kernel_based import RocketClassifier
from sklearn.ensemble import RandomForestClassifier
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from sklearn.model_selection import RandomizedSearchCV
from sktime.classification.compose import SklearnClassifierPipeline

In [ ]:
X_train_Pro, y_train_Pro = load_UCR_UEA_dataset(name='ProximalPhalanxTW', split='train', return_type="numpy2D")
X_test_Pro , y_test_Pro = load_UCR_UEA_dataset(name='ProximalPhalanxTW', split='test', return_type="numpy2D")

#one_hot_encode the labels

# Define the labels
labels = np.unique(y_train_Pro).tolist()

# # One-hot encode the labels
label_map = {label: i for i, label in enumerate(labels)}

# Convert the labels to integers using the label map
y_train_Pro_encoded = [label_map[label] for label in y_train_Pro]
y_test_Pro_encoded = [label_map[label] for label in y_test_Pro]

# Perform one-hot encoding using TensorFlow
y_train_Pro_encoded = tf.one_hot(y_train_Pro_encoded, len(labels))
y_test_Pro_encoded = tf.one_hot(y_test_Pro_encoded, len(labels))

y_train_Pro_encoded = y_train_Pro_encoded.numpy()
y_test_Pro_encoded = y_test_Pro_encoded.numpy()



In [ ]:
X_train_Pro_df = from_2d_array_to_nested(X_train_Pro)
X_test_Pro_df = from_2d_array_to_nested(X_test_Pro)


In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_Pro, y_train_Pro)
rf.score(X_test_Pro, y_test_Pro)

0.7951219512195122

In [ ]:
pip = SklearnClassifierPipeline(classifier=RandomForestClassifier(), transformers=[Rocket(), ExponentTransformer()])

param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [5, 10],
    'Rocket__num_kernels': [100, 500, 1000],
    'ExponentTransformer__power': [1, 2, 3]
}
grid = RandomizedSearchCV(pip, param_grid, n_iter = 3, verbose = 2)


In [ ]:
grid.fit(X_train_Pro_df, y_train_Pro)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ExponentTransformer__power=1, Rocket__num_kernels=1000, classifier__max_depth=5, classifier__n_estimators=100; total time=   3.3s
[CV] END ExponentTransformer__power=1, Rocket__num_kernels=1000, classifier__max_depth=5, classifier__n_estimators=100; total time=   2.8s
[CV] END ExponentTransformer__power=1, Rocket__num_kernels=1000, classifier__max_depth=5, classifier__n_estimators=100; total time=   4.1s
[CV] END ExponentTransformer__power=1, Rocket__num_kernels=1000, classifier__max_depth=5, classifier__n_estimators=100; total time=   2.9s
[CV] END ExponentTransformer__power=1, Rocket__num_kernels=1000, classifier__max_depth=5, classifier__n_estimators=100; total time=   2.8s
[CV] END ExponentTransformer__power=1, Rocket__num_kernels=500, classifier__max_depth=10, classifier__n_estimators=50; total time=   1.5s
[CV] END ExponentTransformer__power=1, Rocket__num_kernels=500, classifier__max_depth=10, classifier__n_est

RandomizedSearchCV(estimator=SklearnClassifierPipeline(classifier=RandomForestClassifier(),
                                                       transformers=[Rocket(),
                                                                     ExponentTransformer()]),
                   n_iter=3,
                   param_distributions={'ExponentTransformer__power': [1, 2, 3],
                                        'Rocket__num_kernels': [100, 500, 1000],
                                        'classifier__max_depth': [5, 10],
                                        'classifier__n_estimators': [50, 100,
                                                                     200]},
                   verbose=2)

In [ ]:
grid.best_estimator_, grid.best_score_

(SklearnClassifierPipeline(classifier=RandomForestClassifier(max_depth=10),
                           transformers=[Rocket(num_kernels=100),
                                         ExponentTransformer(power=2)]),
 0.8299999999999998)

### Task 5: Multivariate time series classification

Time series can be **multivariate**, which means there can be many values (= data points) describing one time point. In this task,we will use one **multivariate** dataset (**BasicMotions**) and try to run one deep learning model and one sktime model to see if those models work well on multivariate time series.

- Use [BasicMotions](https://timeseriesclassification.com/description.php?Dataset=BasicMotions) dataset in the UCR/UEA repository.
- Run **ROCKET** and **the best tensorflow deep learning model from the previous tasks** on BasicMotions. We need to adjust the deep learning model's input layer to handle this multivariate dataset.
- Use sktime's `load_UCR_UEA_dataset` function to perform. We should use each dataset's original train/test splits.
- For the deep learning model, we should transform it using TensorFlow data API (`tf.data`) to manage the dataset and use `shuffle`, `batch`, and `prefetch` functions. This means that we need to create the validation set first. 
- For training, we need to run at least 10 epochs for your deep learning model. For ROCKET, we can keep the default parameter options. 
- Report the test scores of both models on the predefined test set.

In [ ]:
from sktime.classification.kernel_based import RocketClassifier
#import dataset
X_train_BM, y_train_BM = load_UCR_UEA_dataset(name='BasicMotions', split='train')
X_test_BM , y_test_BM = load_UCR_UEA_dataset(name='BasicMotions', split='test')

rocket = RocketClassifier() # SLOW!!!
rocket.fit(X_train_BM, y_train_BM)

RocketClassifier()

In [ ]:
rocket.score(X_test_BM, y_test_BM)

1.0

In [ ]:
#import dataset
X_train_BM, y_train_BM = load_UCR_UEA_dataset(name='BasicMotions', split='train', return_type = 'numpy3D')
X_test_BM , y_test_BM = load_UCR_UEA_dataset(name='BasicMotions', split='test', return_type = 'numpy3D')

#one_hot_encode the labels

# Define the labels
labels = np.unique(y_train_BM).tolist()
# One-hot encode the labels
label_map = {label: i for i, label in enumerate(labels)}
# Convert the labels to integers using the label map
y_train_BM_encoded = [label_map[label] for label in y_train_BM]
y_test_BM_encoded = [label_map[label] for label in y_test_BM]
# Perform one-hot encoding using TensorFlow
y_train_BM_encoded = tf.one_hot(y_train_BM_encoded, len(labels))
y_test_BM_encoded = tf.one_hot(y_test_BM_encoded, len(labels))

y_train_BM_encoded = y_train_BM_encoded.numpy()
y_test_BM_encoded = y_test_BM_encoded.numpy()


In [ ]:
labels, label_map, type(y_train_BM_encoded)

(['badminton', 'running', 'standing', 'walking'],
 {'badminton': 0, 'running': 1, 'standing': 2, 'walking': 3},
 numpy.ndarray)

In [ ]:
#create test,valid,train datasets using from_tensor_slice 
train_ds_BM = tf.data.Dataset.from_tensor_slices((X_train_BM, y_train_BM_encoded))
# Shuffle the training data
idx = np.random.permutation(len(X_train_BM))
X_train_BM, y_train_BM_encoded = X_train_BM[idx], y_train_BM_encoded[idx]
train_ds_BM = train_ds_BM.map(lambda x, y: (tf.cast(x, tf.float32), y))
# create train and valid datasets
valid_split = 0.2
valid_size = int(valid_split * len(X_train_BM))
train_ds_BM = tf.data.Dataset.from_tensor_slices((X_train_BM[valid_size:], y_train_BM_encoded[valid_size:]))
valid_ds_BM = tf.data.Dataset.from_tensor_slices((X_train_BM[:valid_size], y_train_BM_encoded[:valid_size]))
#create test dataset
test_ds_BM = tf.data.Dataset.from_tensor_slices((X_test_BM, y_test_BM_encoded))
test_ds_BM = test_ds_BM.map(lambda x, y: (tf.cast(x, tf.float32), y))

#manage dataset and use shuffle, batch, and prefetch functions
train_ds_BM = train_ds_BM.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
valid_ds_BM = valid_ds_BM.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds_BM = test_ds_BM.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
y_train_BM.shape, y_train_BM_encoded.shape, X_train_BM.shape,type(train_ds_BM), train_ds_BM.cardinality()

((40,),
 (40, 4),
 (40, 6, 100),
 tensorflow.python.data.ops.prefetch_op._PrefetchDataset,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [ ]:
def create_MLP(input_shape, num_label):
    model = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=input_shape),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(500, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(500, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(500, activation='relu'),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(num_label, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
input_shape = (6, 100)
num_label = 4
model_MLP = create_MLP(input_shape, num_label)
#model_MLP.summary()

history = model_MLP.fit(train_ds_BM, validation_data= valid_ds_BM, epochs=10, callbacks=[early_stopping, checkpoint])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 1.9896 - accuracy: 0.2188

1/1 [==============================] - 7s 7s/step - loss: 1.9896 - accuracy: 0.2188 - val_loss: 5.2887 - val_accuracy: 0.3750
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 3.4996 - accuracy: 0.2188

1/1 [==============================] - 1s 1s/step - loss: 3.4996 - accuracy: 0.2188 - val_loss: 4.7106 - val_accuracy: 0.3750
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 2.9018 - accuracy: 0.4062

1/1 [==============================] - 1s 1s/step - loss: 2.9018 - accuracy: 0.4062 - val_loss: 3.4540 - val_accuracy: 0.2500
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 2.1235 - accuracy: 0.4375

1/1 [==============================] - 1s 1s/step - loss: 2.1235 - accuracy: 0.4375 - val_loss: 1.6651 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 1.1119 - accuracy: 0.5625

...

1/1 [==============================] - 2s 2s/step - loss: 0.8367 - accuracy: 0.6250 - val_loss: 0.9163 - val_accuracy: 0.3750
Epoch 7/10
1/1 [==============================] - ETA: 0s - loss: 1.0136 - accuracy: 0.3125

1/1 [==============================] - 2s 2s/step - loss: 1.0136 - accuracy: 0.3125 - val_loss: 0.9632 - val_accuracy: 0.3750
Epoch 8/10
1/1 [==============================] - ETA: 0s - loss: 1.0736 - accuracy: 0.3125

1/1 [==============================] - 1s 1s/step - loss: 1.0736 - accuracy: 0.3125 - val_loss: 0.7016 - val_accuracy: 0.5000
Epoch 9/10
1/1 [==============================] - ETA: 0s - loss: 1.0162 - accuracy: 0.3125

1/1 [==============================] - 1s 1s/step - loss: 1.0162 - accuracy: 0.3125 - val_loss: 0.5192 - val_accuracy: 0.7500
Epoch 10/10
1/1 [==============================] - ETA: 0s - loss: 0.8445 - accuracy: 0.4375

1/1 [==============================] - 1s 1s/step - loss: 0.8445 - accuracy: 0.4375 - val_loss: 0.6007 - val_accuracy: 0.7500


In [ ]:
#model_MLP.evaluate(test_ds_BM)
model_MLP.evaluate(X_test_BM, y_test_BM_encoded)

2/2 [==============================] - 0s 10ms/step - loss: 0.9350 - accuracy: 0.7750


[0.9349850416183472, 0.7749999761581421]